In [1]:
import os
import cfgrib
from pathlib import Path
from datetime import datetime
from time import time


data_store = Path(os.getenv("DATA_STORE_DIR", str(Path("./store")))).absolute()
source = "ecmwf"
model = "enfo"

d = datetime.strptime("20240403", "%Y%m%d")
lead_hour = 36

file_name = f"{d.year}{d.month:02}{d.day:02}000000-{lead_hour}h-enfo-ef.grib2"
file_name

'20240403000000-36h-enfo-ef.grib2'

In [2]:
import xarray as xr
from cgan_ui.show_forecasts import DataArray_from_Dataset_list

mask_bbox = {
    "IGAD": [21, -11.75, 51, 24],
    "KEN": [33.6, -5.84, 43.6, 6.22],
    "ETH": [31.58, 3.12, 47.65, 16.5]
}

def standardize_dataset(ds: xr.DataArray):
    if 'x' in ds.dims and 'y' in ds.dims:
        ds = ds.rename({'x': 'lon', 'y': 'lat'})
    if 'longitude' in ds.dims and 'latitude' in ds.dims:
        ds = ds.rename({'longitude': 'lon', 'latitude': 'lat'})
    return ds

def slice_dataset_by_bbox(dataset, bbox):
    # bbox = [int(item) for item in bbox] # left, bottom, right,
    dataset = dataset.sel(lon=slice(bbox[0], bbox[2]))
    if dataset.lat.values[0] < dataset.lat.values[-1]:
        dataset = dataset.sel(lat=slice(bbox[1], bbox[3]))
    else:
        dataset = dataset.sel(lat=slice(bbox[3], bbox[1]))
    return dataset



def read_dataset(file_path: str) -> list[xr.DataArray]:
    ds = cfgrib.open_datasets(file_path)
    if type(ds) is list:
        arrays = []
        for i in range(len(ds)):
            if 'number' in ds[i].dims:
                arrays.append(standardize_dataset(ds[i]))
        return xr.combine_by_coords(arrays, compat='override')
    return None        


start = time()
xds= read_dataset(f"{data_store}/{source}/{model}/{file_name}")
time() - start

28.650798320770264

In [3]:
from dask.diagnostics import ProgressBar

zarr_file = file_name.replace('grib2','zarr')
write_job = xds.chunk().to_zarr(f"{data_store}/raw/{source}/{model}/{zarr_file}", mode='w', compute=False)
with ProgressBar():
    print(f"Writing to {zarr_file}")
    write_job.compute()

Writing to 20240403000000-36h-enfo-ef.zarr
[########################################] | 100% Completed | 151.76 s


In [4]:
xds = None
zds = xr.open_zarr(f"{data_store}/raw/{source}/{model}/{zarr_file}")
zds

<xarray.Dataset> Size: 23GB
Dimensions:              (number: 50, lat: 721, lon: 1440, isobaricInhPa: 9)
Coordinates: (12/13)
    depthBelowLandLayer  float64 8B ...
    entireAtmosphere     float64 8B ...
    heightAboveGround    float64 8B ...
  * isobaricInhPa        (isobaricInhPa) float64 72B 1e+03 925.0 ... 200.0 50.0
  * lat                  (lat) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * lon                  (lon) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
    ...                   ...
    nominalTop           float64 8B ...
  * number               (number) int64 400B 1 2 3 4 5 6 7 ... 45 46 47 48 49 50
    step                 timedelta64[ns] 8B ...
    surface              float64 8B ...
    time                 datetime64[ns] 8B ...
    valid_time           datetime64[ns] 8B ...
Data variables: (12/37)
    asn                  (number, lat, lon) float32 208MB dask.array<chunksize=(50, 721, 1440), meta=np.ndarray>
    cape                 (number, lat, lon) float32 208MB dask.array<chunksize=(50, 721, 1440), meta=np.ndarray>
    d                    (number, isobaricInhPa, lat, lon) float32 2GB dask.array<chunksize=(50, 9, 721, 1440), meta=np.ndarray>
    d2m                  (number, lat, lon) float32 208MB dask.array<chunksize=(50, 721, 1440), meta=np.ndarray>
    gh                   (number, isobaricInhPa, lat, lon) float32 2GB dask.array<chunksize=(50, 9, 721, 1440), meta=np.ndarray>
    lsm                  (number, lat, lon) float32 208MB dask.array<chunksize=(50, 721, 1440), meta=np.ndarray>
    ...                   ...
    u100                 (number, lat, lon) float32 208MB dask.array<chunksize=(50, 721, 1440), meta=np.ndarray>
    v                    (number, isobaricInhPa, lat, lon) float32 2GB dask.array<chunksize=(50, 9, 721, 1440), meta=np.ndarray>
    v10                  (number, lat, lon) float32 208MB dask.array<chunksize=(50, 721, 1440), meta=np.ndarray>
    v100                 (number, lat, lon) float32 208MB dask.array<chunksize=(50, 721, 1440), meta=np.ndarray>
    vo                   (number, isobaricInhPa, lat, lon) float32 2GB dask.array<chunksize=(50, 9, 721, 1440), meta=np.ndarray>
    w                    (number, isobaricInhPa, lat, lon) float32 2GB dask.array<chunksize=(50, 9, 721, 1440), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_edition:            2
    GRIB_subCentre:          0
    institution:             European Centre for Medium-Range Weather Forecasts

In [ ]:
for key in mask_bbox.keys():
    sliced = slice_dataset_by_bbox(zds, mask_bbox[key])
    out_dir = Path(f"{data_store}/interim/{key}/{source}/{model}")
    if not out_dir.exists():
        out_dir.mkdir(parents=True)
    sliced.to_netcdf(out_dir / file_name.replace('grib2','nc'))
